# This notebook extracts and saves preliminary data from https://comtrade.un.org/

In [1]:
import pandas as pd
import numpy
import requests
import json
import codecs
import pickle
from collections import defaultdict
from datetime import datetime
from dateutil.relativedelta import relativedelta
import time
from retrying import retry


In [2]:
# Get country codes

country_codes = requests.get('https://comtrade.un.org/Data/cache/reporterAreas.json')
country_codes = country_codes.text.encode().decode('utf-8-sig') 
country_codes = json.loads(country_codes)["results"]

with open('Input_datasets/country_codes_old.pickle', 'wb') as handle:
    pickle.dump(country_codes, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
# Get monthly data availability for all countries
data_availability = requests.get('https://comtrade.un.org/api/refs/da/view?type=C&freq=M')

country_availability = defaultdict(list)

for i in data_availability.json():
    country_availability[i['r']].append(datetime.strptime(i['ps'], '%Y%m').date())

In [4]:
# Define datelist that will be the timeframe for our analysis
startDate = '2011-01'
endDate = '2020-12'

start = datetime.strptime(startDate, '%Y-%m').date()
end = datetime.strptime(endDate, '%Y-%m').date()

datelist = []

while start < end:
    datelist.append(start)
    start += relativedelta(months=1)

In [5]:
# Define how many months can be missing from a given country in the timeframe
# Obtain list of countries to which we have sufficient data
max_missing_months = 0
countries_with_sufficient_data = []

for country, dates in country_availability.items():
    missing_months = 0
    for j in datelist:
        if j not in dates:
            missing_months+=1
    if missing_months<=max_missing_months:
        countries_with_sufficient_data.append(country)

len(countries_with_sufficient_data)

67

In [6]:
#Export countries in batches of 5
countries_string_in_fives = []
for i in range(int(len(countries_with_sufficient_data)/5)+1):
    countries_string_in_fives.append("%2C".join(countries_with_sufficient_data[i*5:i*5+5]))

exported_data_list = []
for element1 in countries_string_in_fives:
    for element2 in countries_string_in_fives:
        # max - maximum records returned (default = 500) A valid integer in the range (Guest: [1, 100000], Authenticated: [1, account limit*], NOTE: Maximum limit is 250,000).
        # type - trade data type (default = any) Valid values: C Commodities (merchandise trade data), S Services (trade in services data)
        # freq - (default = any) data set frequency: Valid values: A Annual, M Monthly
        # px - classification (default = any): Trade data classification scheme. HS Harmonized System (HS), as reported (e.g. if data was originally submitted to UN Comtrade in HS1996 then HS1996 is displayed)
        # ps - time period (default = any): Depending on freq, time period can take either YYYY or YYYYMM
        # r - (default = any) reporting area: the area that reported the trade to UNSD.
        # p - partner area (default = all): partner area. The area receiving the trade, based on the reporting areas data.
        # rg - trade regime / trade flow (default = all): The most common area 1 (imports) and 2 (exports)
        # cc - classification code (default = AG2): a commodity code valid in the selected classification. Full lists of codes for each classification are linked to above under the px parameter.
        #                                         Some codes are valid in all classifications: TOTAL - Total trade between reporter and partner, no detail breakdown.
        exported_data = requests.get(f'https://comtrade.un.org/api/get?max=100000&type=C&freq=M&px=HS&ps=all&r={element1}&p={element2}&rg=2&cc=TOTAL')
        if exported_data.status_code==200:
            exported_data_list.append(exported_data)
        else:
            print(element1, element2)
            break
        time.sleep(40)

with open('Input_datasets/exported_data_list_old.pickle', 'wb') as handle:
    pickle.dump(exported_data_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
len(exported_data_list)

196

In [10]:
final_values = []

for export in exported_data_list:
    exported_data = export.json()['dataset']
    for entry in exported_data:
        final_values.append([entry['rtCode'], entry['ptCode'], entry['period'], entry['TradeValue']])

final_df = pd.DataFrame(final_values, columns=['From', 'To', 'Period', 'Value'])

datelist_string = [int(i.strftime('%Y%m')) for i in datelist]
final_df = final_df[final_df['Period'].isin(datelist_string)].copy()

final_df.to_csv('Input_datasets/trade_data_old.csv')

# Using new API and imports on monthly level

In [15]:
# Get country codes
hdr ={
    # Request headers
    'Cache-Control': 'no-cache',
    'Ocp-Apim-Subscription-Key': '66ced423d77f4e4bb92ea4c51a8b5c5f',
    }

country_codes = requests.get('https://comtradeapi.un.org/files/v1/app/reference/Reporters.json')
country_codes = country_codes.text.encode().decode('utf-8-sig') 
country_codes = json.loads(country_codes)["results"]

with open('Input_datasets/country_codes_new.pickle', 'wb') as handle:
    pickle.dump(country_codes, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [47]:
# Get monthly data availability for all countries

# typeCode: C (commodity)
# freqCode: M (monthly)
# clCode: HS (Trade data classification scheme. HS Harmonized System)

data_availability = requests.get('https://comtradeapi.un.org/data/v1/getDa/C/M/HS', headers=hdr)

country_availability = defaultdict(list)

for i in data_availability.json()['data']:
    country_availability[i['reporterCode']].append(datetime.strptime(str(i['period']), '%Y%m').date())

In [48]:
# Define datelist that will be the timeframe for our analysis
startDate = '2011-01'
endDate = '2020-12'

start = datetime.strptime(startDate, '%Y-%m').date()
end = datetime.strptime(endDate, '%Y-%m').date()

datelist = []

while start < end:
    datelist.append(start)
    start += relativedelta(months=1)

In [54]:
# Define how many months can be missing from a given country in the timeframe
# Obtain list of countries to which we have sufficient data
max_missing_months = 0
countries_with_sufficient_data = []

for country, dates in country_availability.items():
    missing_months = 0
    for j in datelist:
        if j not in dates:
            missing_months+=1
    if missing_months<=max_missing_months:
        countries_with_sufficient_data.append(str(country))

len(countries_with_sufficient_data)

64

In [85]:
datelist_string = [str(i.strftime('%Y%m')) for i in datelist]
dates = ','.join(datelist_string)

In [89]:
#Export countries in batches of 5 with new API
countries_string_in_fives = []
for i in range(int(len(countries_with_sufficient_data)/5)+1):
    countries_string_in_fives.append(",".join(countries_with_sufficient_data[i*5:i*5+5]))

exported_data_list = []
for element1 in countries_string_in_fives:
    for element2 in countries_string_in_fives:
        exported_data = requests.get(f'https://comtradeapi.un.org/data/v1/get/C/M/HS?reporterCode={element1}&period={dates}&partnerCode={element2}&cmdCode=total&flowCode=M', headers=hdr)
        if exported_data.status_code==200:
            exported_data_list.append(exported_data)
        else:
            print(element1, element2)
            break
        time.sleep(40)

with open('Input_datasets/exported_data_list_new_monthly_import.pickle', 'wb') as handle:
    pickle.dump(exported_data_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [90]:
#At the most basic data record, up to three trade values could be presented according to following rules:
#a) CIF value for imports would be assigned to “CIF Value”
#b) When FOB value is reported for imports, it would be allocated to “FOB Value”
#c) FOB value for exports would be allocated to “FOB Value”
#For users’ convenience, a third (“primary”) trade value is generated by applying IMTS 2010 recommendation on valuation: if available CIF otherwise FOB for imports; FOB for exports, respectively.

final_values = []

for export in exported_data_list:
    exported_data = export.json()['data']
    for entry in exported_data:
        final_values.append([entry['reporterCode'], entry['partnerCode'], entry['period'], entry['primaryValue']])

final_df = pd.DataFrame(final_values, columns=['From', 'To', 'Period', 'Value'])

#datelist_string = [int(i.strftime('%Y%m')) for i in datelist]
#final_df = final_df[final_df['Period'].isin(datelist_string)].copy()

final_df.to_csv('Input_datasets/trade_data_new_monthly_import.csv')

# Using new API and imports on yearly level

In [2]:
# Get country codes
hdr ={
    # Request headers
    'Cache-Control': 'no-cache',
    'Ocp-Apim-Subscription-Key': '66ced423d77f4e4bb92ea4c51a8b5c5f',
    }

country_codes = requests.get('https://comtradeapi.un.org/files/v1/app/reference/Reporters.json')
country_codes = country_codes.text.encode().decode('utf-8-sig') 
country_codes = json.loads(country_codes)["results"]

#with open('Input_datasets/country_codes_new.pickle', 'wb') as handle:
#    pickle.dump(country_codes, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
# Get yearly data availability for all countries

# typeCode: C (commodity)
# freqCode: A (annual)
# clCode: HS (Trade data classification scheme. HS Harmonized System)

data_availability = requests.get('https://comtradeapi.un.org/data/v1/getDa/C/A/HS', headers=hdr)

country_availability = defaultdict(list)

for i in data_availability.json()['data']:
    country_availability[i['reporterCode']].append(datetime.strptime(str(i['period']), '%Y').date())

In [4]:
# Define datelist that will be the timeframe for our analysis
startDate = '1995'
endDate = '2019'

start = datetime.strptime(startDate, '%Y').date()
end = datetime.strptime(endDate, '%Y').date()

datelist = []

while start <= end:
    datelist.append(start)
    start += relativedelta(years=1)

In [5]:
# Define how many years can be missing from a given country in the timeframe
# Obtain list of countries to which we have sufficient data
max_missing_years = 0
countries_with_sufficient_data = []

for country, dates in country_availability.items():
    missing_years = 0
    for j in datelist:
        if j not in dates:
            missing_years+=1
    if missing_years<=max_missing_years:
        countries_with_sufficient_data.append(str(country))

len(countries_with_sufficient_data)

86

In [6]:
datelist_string = [str(i.strftime('%Y')) for i in datelist]
dates = ','.join(datelist_string)

In [7]:
#Export countries in batches of 5 with new API
countries_string_in_fives = []
for i in range(int(len(countries_with_sufficient_data)/5)+1):
    countries_string_in_fives.append(",".join(countries_with_sufficient_data[i*5:i*5+5]))

def retry_if_connection_error(exception):
    return isinstance(exception, ConnectionError)

def retry_if_status_code_not_200(result):
    return result.status_code!=200

# if exception retry with 2 second wait  
@retry(retry_on_exception=retry_if_connection_error, retry_on_result=retry_if_status_code_not_200, wait_fixed=2000)
def safe_request(url, **kwargs):
    return requests.get(url, **kwargs)

exported_data_list = []
for element1 in countries_string_in_fives:
    for element2 in countries_string_in_fives:
        exported_data = safe_request(f'https://comtradeapi.un.org/data/v1/get/C/A/HS?reporterCode={element1}&period={dates}&partnerCode={element2}&cmdCode=total&flowCode=M&breakdownMode=classic', headers=hdr)
        exported_data_list.append(exported_data)
        #try:
        #    exported_data = requests.get(f'https://comtradeapi.un.org/data/v1/get/C/A/HS?reporterCode={element1}&period={dates}&partnerCode={element2}&cmdCode=total&flowCode=M', headers=hdr)
        #    if exported_data.status_code==200:
        #        exported_data_list.append(exported_data)
        #    else:
        #        print(element1, element2)
        #        break
        #except requests.exceptions.ConnectionError:
        #    print("Connection error")
        #    print(f'https://comtradeapi.un.org/data/v1/get/C/A/HS?reporterCode={element1}&period={dates}&partnerCode={element2}&cmdCode=total&flowCode=M')
        time.sleep(40)

with open('exported_data_list_new_annual_import.pickle', 'wb') as handle:
    pickle.dump(exported_data_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
#At the most basic data record, up to three trade values could be presented according to following rules:
#a) CIF value for imports would be assigned to “CIF Value”
#b) When FOB value is reported for imports, it would be allocated to “FOB Value”
#c) FOB value for exports would be allocated to “FOB Value”
#For users’ convenience, a third (“primary”) trade value is generated by applying IMTS 2010 recommendation on valuation: if available CIF otherwise FOB for imports; FOB for exports, respectively.

final_values = []
final_values_full = []

for export in exported_data_list:
    exported_data = export.json()['data']
    for entry in exported_data:
        final_values.append([entry['reporterCode'], entry['partnerCode'], entry['period'], entry['primaryValue']])
        final_values_full.append(entry)

final_df = pd.DataFrame(final_values, columns=['Reporter', 'Partner', 'Period', 'Value'])
final_df_full = pd.DataFrame(final_values_full)

#datelist_string = [int(i.strftime('%Y%m')) for i in datelist]
#final_df = final_df[final_df['Period'].isin(datelist_string)].copy()

final_df.to_csv('trade_data_new_annual_import.csv', index=False)
final_df_full.to_csv('trade_data_new_annual_import_full_columns.csv', index=False)

In [11]:
final_df.shape

(159723, 4)

In [13]:
len(set(list(zip(final_df.Reporter, final_df.Partner, final_df.Period))))

159723

In [14]:
# Add zero flows
country_codes = set(final_df["Reporter"]) | set(final_df["Partner"])
year_dates = set(final_df["Period"])
available_data = set(zip(final_df.Reporter, final_df.Partner, final_df.Period))
all_data = set([(origin, destination, time_period) for origin in country_codes for destination in country_codes for time_period in year_dates])
zero_flow_data = list(all_data - available_data)

zero_flow_df = pd.DataFrame(zero_flow_data, columns =['Reporter', 'Partner', 'Period'])
zero_flow_df['Value'] = 0

final_df = pd.concat([final_df, zero_flow_df])
final_df = final_df.sort_values(by=['Reporter', 'Partner', 'Period'])

final_df['Value'] = final_df['Value'].astype('int64')

final_df.to_csv('trade_data_new_annual_import_zero_padded.csv', index=False)

In [28]:
final_df.shape

(184900, 4)

# Export country names that have annual trade data

In [19]:
country_codes_dict = {}

for element in country_codes:
    country_codes_dict[element['id']] = element['text']

country_names_to_work_with = []
for country in countries_with_sufficient_data:
    country_names_to_work_with.append([country, country_codes_dict[int(country)]])

In [23]:
country_names_to_work_with_df = pd.DataFrame(country_names_to_work_with, columns=['ISO_3166-1_numeric_code', 'Country_name'])
country_names_to_work_with_df.to_csv('Input_datasets/country_names_with_annual_trade_data.csv')

In [24]:
country_names_to_work_with_df

,ISO_3166-1_numeric_code,Country_name
0,32,Argentina
1,36,Australia
2,40,Austria
3,68,Bolivia (Plurinational State of)
4,76,Brazil
...,...,...
81,834,United Rep. of Tanzania
82,842,USA
83,854,Burkina Faso
84,858,Uruguay


# Ad-hoc, messy stuff

In [3]:
import pickle

with open('Input_datasets/exported_data_list_new_monthly_import.pickle', 'rb') as handle:
    exported_data_list = pickle.load(handle)


In [17]:
exported_data_list[0].json()['data'][0]['primaryValue']

24649531.515

In [70]:
export.json()['data'][0].keys()

dict_keys(['typeCode', 'freqCode', 'refPeriodId', 'refYear', 'refMonth', 'period', 'reporterCode', 'reporterISO', 'reporterDesc', 'flowCode', 'flowDesc', 'partnerCode', 'partnerISO', 'partnerDesc', 'partner2Code', 'partner2ISO', 'partner2Desc', 'classificationCode', 'classificationSearchCode', 'isOriginalClassification', 'cmdCode', 'cmdDesc', 'aggrLevel', 'isLeaf', 'customsCode', 'customsDesc', 'mosCode', 'motCode', 'motDesc', 'qtyUnitCode', 'qtyUnitAbbr', 'qty', 'isQtyEstimated', 'altQtyUnitCode', 'altQtyUnitAbbr', 'altQty', 'isAltQtyEstimated', 'netWgt', 'isNetWgtEstimated', 'grossWgt', 'isGrossWgtEstimated', 'cifvalue', 'fobvalue', 'primaryValue', 'legacyEstimationFlag', 'isReported', 'isAggregate'])

In [1]:
final_df['Period'].unique()

NameError: name 'final_df' is not defined

In [93]:
len(final_df['From'].unique())

64

In [94]:
len(final_df['To'].unique())

63

In [95]:
set(final_df['From'].unique()) - set(final_df['To'].unique())

{97}